# Anleitung zur Erstellung eines Computational Essays in Jupyter Notebook

## Zielgruppe
Diese Anleitung richtet sich an Studierende der digitalen Geschichtswissenschaft, die einen **Computational Essay** in Form eines Jupyter Notebooks erstellen und einreichen möchten.


## Voraussetzungen

Bevor Sie mit der Konvertierung beginnen können, benötigen Sie:

1. **Python** (Version 3.8 oder höher)
2. **Pandoc** (für die Dokumentkonvertierung)
3. Die Python-Pakete:
   - nbformat
   - jupyter
   - os
   - re
   - subprocess

## Installation der Voraussetzungen

### 1. Pandoc Installation
- Windows: Laden Sie den Installer von [pandoc.org/installing.html](https://pandoc.org/installing.html) herunter
- Mac: `brew install pandoc`
- Linux: `sudo apt-get install pandoc`

### 2. Python-Pakete Installation
```bash
pip install nbformat jupyter
```

## Nutzungsmöglichkeiten

Sie haben zwei Möglichkeiten, diesen Konverter zu nutzen:

**Option 1: Einzelne Notebook-Datei**

1. Laden Sie die Datei [word_md_notebook.ipynb](https://github.com/Digital-History-Berlin/Computational-Essays/blob/main/Konvertierungen/word_md_notebook.ipynb) herunter
2. Erstellen Sie einen neuen Ordner für Ihr Projekt
3. Legen Sie das Notebook in diesem Ordner ab
4. Erstellen Sie einen Unterordner input für Ihre Word-Dokumente

**Option 2: Repository klonen**

1. Klonen Sie das Repository:
    ```bash
    git clone https://github.com/Digital-History-Berlin/Computational-Essays.git
    ```
2. Navigieren Sie zum Projektordner
3. Folgen Sie den weiteren Anweisungen

Ein beispielhaftes Word-Dokument ist ebenfalls bereits im Repository enthalten.

## Verwendung

1. Legen Sie Ihr Word-Dokument im Projektordner ab
2. Öffnen Sie das Notebook in Jupyter
3. Passen Sie die Dateipfade an:

```python
docx_path = "IhrDokument.docx"
output_md = "Output.md"
images_folder = "images"
```

4. Führen Sie alle Zellen nacheinander aus



## Konvertierung von Word-Dokumenten zu Markdown mit Bildextraktion

### Was macht dieser Schritt?

Dieses Skript dient der Konvertierung von **Word-Dokumenten (.docx)** in **Markdown**-Format. Dabei werden alle im Dokument enthaltenen Bilder automatisch extrahiert und in einem separaten Ordner gespeichert. Zusätzlich sorgt das Skript dafür, dass Tabellen, Fußnoten und andere akademische Inhalte in ein Markdown-kompatibles Format überführt werden. 

### Warum ist dieser Schritt notwendig?

Die Konvertierung von `.docx` zu Markdown ist ein wichtiger Zwischenschritt, um die Inhalte eines Textdokuments in ein Jupyter Notebook zu integrieren. Markdown ist das bevorzugte Format für Notebooks, da es sowohl Klartext als auch strukturierte Inhalte wie Tabellen, Bilder und Code unterstützt. Die Bildextraktion stellt sicher, dass eingebettete Medien korrekt referenziert und später im Jupyter Notebook dargestellt werden können.

### Wie funktioniert es?

1. **Dateiüberprüfung und Ordnererstellung**:
   - Es wird sichergestellt, dass der Zielordner für Bilder existiert oder bei Bedarf erstellt wird.

2. **Pandoc-Kommando**:
   - Mithilfe von **Pandoc**, einem leistungsstarken Konvertierungstool, wird das `.docx`-Dokument in Markdown umgewandelt.
   - GitHub-Flavored Markdown (GFM) wird als Ziel gewählt, um saubere Tabellen und bessere Kompatibilität zu gewährleisten.
   - Bilder werden mit dem Parameter `--extract-media` automatisch extrahiert und im angegebenen Ordner gespeichert.

3. **Speicherung und Ausgabe**:
   - Das Markdown-Dokument wird im definierten Pfad gespeichert, und die extrahierten Bilder werden in den Bildordner verschoben.

### Vorteile dieses Workflows

- **Saubere Strukturierung**:
  Tabellen und Inhalte werden in einem für Jupyter Notebook gut lesbaren Format erstellt.
- **Automatische Bildextraktion**:
  Bilder werden automatisch verarbeitet und in Markdown korrekt referenziert, was manuelle Nacharbeit reduziert.
- **Reproduzierbarkeit**:
  Die automatisierte Konvertierung minimiert Fehler und sorgt für konsistente Ergebnisse.


Nach Abschluss der Konvertierung finden Sie:

Eine Markdown-Datei (sample.md) mit allen Textinhalten.
Alle eingebetteten Bilder im Ordner images.

In [1]:
import os
import re
import subprocess

In [ ]:
def convert_docx_to_md_with_images(docx_path, output_md, images_folder):
    """
    Konvertierung von .docx Datei zu Markdown und extrahiert die Bilder automatisch in einen Ordner.
    """
    # Ensure the images folder exists
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)

    # Use Pandoc for text conversion with image extraction
    pandoc_command = [
        "pandoc",
        docx_path,
        "-f", "docx",
        "-t", "gfm",  # GitHub-Flavored Markdown for cleaner tables
        "--wrap=none",
        f"--extract-media={images_folder}",  # Extract media to the specified folder
        "-o", output_md
    ]
    
    try:
        subprocess.run(pandoc_command, check=True)
        print(f"Conversion completed. Markdown saved to {output_md}. Images saved to {images_folder}.")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")

# Example usage
docx_path = "Sample_Academic_Document.docx"  # Path to the input Word document
output_md = "Sample_Markdown.md"  # Path to the output Markdown file
images_folder = "images"  # Folder to store extracted images

convert_docx_to_md_with_images(docx_path, output_md, images_folder)


Conversion completed. Markdown saved to Sample_Markdown.md. Images saved to images.


## Konvertierung von Markdown in ein Jupyter Notebook

### Warum ist dieser Schritt notwendig?

Dieser Schritt ist erforderlich, um das vorbereitete Markdown-Dokument in ein **Jupyter Notebook** umzuwandeln. Das Notebook ermöglicht die interaktive Arbeit mit Text, Code und Visualisierungen und ist ideal für Computational Essays geeignet. 

Markdown allein kann die Funktionalität eines Notebooks nicht bieten, daher müssen:
- Markdown-Zellen und ggf. Code-Zellen identifiziert werden,
- Bilder eingebunden werden,
- Metadaten hinzugefügt werden, um das Notebook ausführbar zu machen.

---

### Was passiert in diesem Schritt?

1. **Verarbeitung der Fußnoten**:
   - Fußnoten werden aus dem Ende des Dokuments extrahiert und an die Absätze angehängt, auf die sie sich beziehen.
   - Dies stellt sicher, dass die Fußnoten beim Aufteilen in Zellen an der richtigen Stelle verbleiben.

2. **Einfügen von Zellmarkierungen**:
   - Zellmarkierungen (`<!-- #region -->` und `<!-- #endregion -->`) werden im Markdown-Dokument eingefügt, um festzulegen, welche Abschnitte als einzelne Zellen im Notebook dargestellt werden sollen.
   - Codeblöcke werden separat erkannt und entsprechend markiert.

3. **Konvertierung zu einem Notebook**:
   - Das Markdown-Dokument wird in Zellen aufgeteilt (Markdown- und Code-Zellen).
   - Bilder werden mit relativen Pfaden korrekt referenziert.
   - Metadaten werden hinzugefügt, um sicherzustellen, dass das Notebook in Jupyter korrekt ausgeführt werden kann.

---

### Wo muss der Benutzer Änderungen vornehmen?

- **Dateipfade anpassen**:
  - Ändern Sie die Pfade für die Eingabedateien (`sample.md`) und Ausgabedateien (`output_notebook.ipynb`), um mit Ihren spezifischen Dateien zu arbeiten.
  - Beispiel:
    markdown_file = "mein_markdown.md"
    notebook_file = "mein_notebook.ipynb"
    images_folder = "meine_bilder"

- **Format prüfen**:
  - Stellen Sie sicher, dass das Markdown-Dokument korrekt formatiert ist (z. B. gültige Codeblöcke und Bilder mit `![Beschreibung](pfad)`).

---

### Vorteile dieses Workflows

- **Automatisiert**: Der Prozess automatisiert die Umwandlung, wodurch manuelle Arbeit minimiert wird.
- **Flexibel**: Der Benutzer kann Änderungen an den Markdown-Inhalten vornehmen, bevor die Konvertierung erfolgt.
- **Kompatibel**: Das erzeugte Notebook ist mit Jupyter kompatibel und bereit zur interaktiven Nutzung.

In [3]:
import nbformat as nbf

In [ ]:
def process_footnotes(input_file, output_file):
    """
    Verschiebt Fußnoten in den Absatz, in dem sie referenziert werden.
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Step 1: Extract footnote definitions from the end
    footnote_definitions = {}
    footnote_def_pattern = re.compile(r'^\[\^([^\]]+)\]:\s+(.*)', re.MULTILINE)
    footnotes = footnote_def_pattern.findall(content)

    # Build a dictionary of footnote definitions
    for footnote_id, footnote_text in footnotes:
        footnote_definitions[footnote_id] = footnote_text.strip()

    # Remove footnote definitions from the content
    content = footnote_def_pattern.sub('', content).strip()

    # Step 2: Place footnote definitions after their references
    footnote_ref_pattern = re.compile(r'\[\^([^\]]+)\]')
    paragraphs = content.split('\n\n')
    included_footnotes = set()
    processed_paragraphs = []

    for paragraph in paragraphs:
        refs_in_paragraph = footnote_ref_pattern.findall(paragraph)
        if refs_in_paragraph:
            for ref in refs_in_paragraph:
                if ref in footnote_definitions and ref not in included_footnotes:
                    footnote_def = f"\n\n[^{ref}]: {footnote_definitions[ref]}"
                    paragraph += footnote_def
                    included_footnotes.add(ref)
        processed_paragraphs.append(paragraph)

    # Reconstruct the content
    new_content = '\n\n'.join(processed_paragraphs)

    # Step 3: Write the modified content to the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(new_content)

def insert_cell_markers(input_file, output_file):
    """
    Fügt Zellmarker zu Markdown hinzu, um die Aufteilung in Jupyter Notebook-Zellen vorzubereiten.
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Insert cell markers before headings and code blocks
    content = re.sub(r'(^#+ .*$)', r'<!-- #endregion -->\n\n<!-- #region -->\n\1', content, flags=re.MULTILINE)
    content = re.sub(r'(```.*?```)', r'<!-- #endregion -->\n\n<!-- #region -->\n\1\n\n<!-- #endregion -->', content, flags=re.DOTALL)

    # Add starting and ending cell markers if missing
    if not content.startswith('<!-- #region -->'):
        content = '<!-- #region -->\n' + content
    if not content.rstrip().endswith('<!-- #endregion -->'):
        content += '\n<!-- #endregion -->'

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)

def convert_markdown_to_notebook(markdown_file, notebook_file, images_folder):
    """
    Konvertiert Markdown mit Zellmarkierungen in eine Jupyter Notebook-Datei.
    """
    with open(markdown_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the Markdown into sections based on cell markers
    sections = re.split(r'<!-- #region -->|<!-- #endregion -->', content)
    notebook = nbf.v4.new_notebook()

    # Process each section
    for section in sections:
        section = section.strip()
        if not section:
            continue

        # Detect code or Markdown
        if section.startswith("```") and section.endswith("```"):
            # Code cell
            code_content = section.strip("```").strip()
            notebook.cells.append(nbf.v4.new_code_cell(code_content))
        else:
            # Markdown cell
            # Adjust image references to use the relative path
            section = re.sub(r'!\[(.*?)\]\((.*?)\)', rf'![\1]({images_folder}/\2)', section)
            notebook.cells.append(nbf.v4.new_markdown_cell(section))

    # Write the notebook to a file
    with open(notebook_file, 'w', encoding='utf-8') as f:
        nbf.write(notebook, f)

# Example Usage
markdown_file = "Sample_Markdown.md"
notebook_file = "Sample_Notebook.ipynb"
images_folder = "images"

# Step 1: Process footnotes
process_footnotes(markdown_file, markdown_file)

# Step 2: Insert cell markers
insert_cell_markers(markdown_file, markdown_file)

# Step 3: Convert to Jupyter Notebook
convert_markdown_to_notebook(markdown_file, notebook_file, images_folder)

print(f"Notebook created: {notebook_file}")


Notebook created: Sample_Notebook.ipynb
